## PYTHON CODE -> Segmenting and Clustering Neighborhoods in Toronto

### Installing & Importing Libraries

In [23]:
#IMPORTING NUMPY
import numpy as np 
#IMPORTING PANDAS
import pandas as pd
#IMPORTING JSON
import json
#INSTALLING & IMPORTING GEOPY
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
#IMPORTING REQUESTS
import requests 
from pandas.io.json import json_normalize
#IMPORTING MATPLOTLIB
import matplotlib.cm as cm
import matplotlib.colors as colors
#IMPORTING KMEANS
from sklearn.cluster import KMeans
#INSTALLING & IMPORING FOLIUM
!conda install -c conda-forge folium=0.5.0 --yes
import folium
#IMPORTING BEAUTIFULSOUP
from bs4 import BeautifulSoup
#IMPORTING LXML
import lxml
from selenium import webdriver
print('Libraries installed & imported.')

Libraries installed & imported.


### Importing Toronto Table 

In [24]:
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
table = pd.read_html(url)
data = table[0]
data.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Data Cleaning

In [25]:
#Removing 'Not assigned' Values
data = data[data.Borough!='Not assigned']
number_of_rows = data.shape[0]
#Replacing unknown neighbourhood values with borough name
for i in range(0,number_of_rows):
    if data.iloc[i][2] == 'Not assigned':
        data.iloc[i][2] = data.iloc[i][1]
#Grouping similar postcodes & boroughs into single row
df = data.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df.rename(columns = {'Postcode':'Postal Code'},inplace= True)
df.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [26]:
df.shape

(103, 3)

### Loading Co-ordinates CSV

In [27]:
coordinates = pd.read_csv('Geospatial_Coordinates.csv')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Mergining Co-ordinates with Clean Data

In [28]:
toronto = df.merge(coordinates,how='right')
toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## CREATING A MAP

### Finding Co-ordinates of Toronto

In [29]:
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Creating a map using Folium

In [30]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
#Adding Markers
for lat, lng, borough, neighbourhood in zip(toronto['Latitude'],toronto['Longitude'],toronto['Borough'],toronto['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)  
map_toronto

### Listing Boroughs

In [31]:
toronto['Borough'].value_counts()

North York          24
Downtown Toronto    18
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
York                 5
East Toronto         5
East York            5
Queen's Park         1
Mississauga          1
Name: Borough, dtype: int64

### Let us choose Downtown Toronto

In [32]:
DT = toronto[toronto['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
DT.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


### Location Downtown Toronto

In [33]:
address = 'Downtown Toronto, ON'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.655115, -79.380219.


In [34]:
map_DT = folium.Map(location=[latitude, longitude], zoom_start=13)
for lat, lng, label in zip(DT['Latitude'],DT['Longitude'],DT['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.5,
        parse_html=False).add_to(map_DT)  
    
map_DT

### Foursquare API Details

In [35]:
CLIENT_ID = 'SPJ5ZZH2JCBKLK3Y5HF51JBNPK5GGGPI2J43MDND3Z3MFD2L' # your Foursquare ID
CLIENT_SECRET = 'Y1J35FIYYFC1LUW3ATM0TVBTB1E3TVY4ALG2F2FZXSXYNCFB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

### Exploring Neighbourhoods in Downtown Toronto

In [36]:
def getNearbyVenues(names, latitudes, longitudes, radius=500,LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        #API ->Request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        #Getting the values
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [37]:
DT_venues = getNearbyVenues(names=DT['Neighbourhood'],
                                   latitudes=DT['Latitude'],
                                   longitudes=DT['Longitude']
                                  )

Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie


In [38]:
print(DT_venues.shape)
DT_venues.head()

(1301, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Mooredale House,43.678631,-79.380091,Building
1,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
2,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
3,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
4,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail


In [39]:
DT_onehot = pd.get_dummies(DT_venues[['Venue Category']], prefix="", prefix_sep="")
DT_onehot['Neighbourhood'] = DT_venues['Neighbourhood'] 
fixed_columns = [DT_onehot.columns[-1]] + list(DT_onehot.columns[:-1])
DT_onehot = DT_onehot[fixed_columns]
DT_grouped = DT_onehot.groupby('Neighbourhood').mean().reset_index()
DT_grouped

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide, King, Richmond",0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.00,0.010000,0.010000,0.00000,0.02,0.0000,0.000000,0.0000,0.000000,0.020000,0.000000,0.040000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.01,0.020000,0.000000,0.000000,0.01,0.030000,0.01000,0.000000,0.050000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.080000,0.000000,0.000000,0.00,0.01,0.000000,0.000000,0.020000,0.0000,0.030000,0.000000,0.00,0.01000,0.010000,0.010000,0.000000,0.00,0.000000,0.000000,0.00000,0.00,0.000000,0.00,0.000000,0.010000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00000,0.01,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.020000,0.000000,0.000000,0.01,0.000000,0.01,0.000000,0.010000,0.000000,0.020000,0.010000,0.000000,0.00000,0.000000,0.000000,0.00,0.00000,0.00,0.00,0.000000,0.030000,0.00,0.00,0.010000,0.010000,0.00000,0.000000,0.010000,0.010000,0.010000,0.000000,0.010000,0.000000

In [40]:
num_top_venues = 5
for hood in DT_grouped['Neighbourhood']:
    print(hood.upper())
    temp = DT_grouped[DT_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

ADELAIDE, KING, RICHMOND
            venue  freq
0     Coffee Shop  0.08
1            Café  0.05
2      Steakhouse  0.04
3             Bar  0.04
4  Cosmetics Shop  0.03


BERCZY PARK
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2      Farmers Market  0.04
3  Italian Restaurant  0.04
4                Café  0.04


CN TOWER, BATHURST QUAY, ISLAND AIRPORT, HARBOURFRONT WEST, KING AND SPADINA, RAILWAY LANDS, SOUTH NIAGARA
              venue  freq
0    Airport Lounge  0.13
1   Airport Service  0.13
2  Airport Terminal  0.13
3             Plane  0.07
4   Harbor / Marina  0.07


CABBAGETOWN, ST. JAMES TOWN
                venue  freq
0         Coffee Shop  0.07
1            Pharmacy  0.04
2  Italian Restaurant  0.04
3         Pizza Place  0.04
4                Park  0.04


CENTRAL BAY STREET
                       venue  freq
0                Coffee Shop  0.14
1                       Café  0.06
2  Middle Eastern Restaurant  0.05
3         Italian Restaur

### K-Means Clustering

In [41]:
k = 5
DT_grouped_clustering = DT_grouped.drop('Neighbourhood', 1)
#Clustering
kmeans = KMeans(n_clusters=k, random_state=0).fit(DT_grouped_clustering)
kmeans.labels_[0:10] 

array([0, 0, 3, 0, 0, 2, 4, 0, 0, 0])

In [42]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [43]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = DT_grouped['Neighbourhood']

for ind in np.arange(DT_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(DT_grouped.iloc[ind, :], num_top_venues)
neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Bar,American Restaurant,Thai Restaurant,Restaurant,Burger Joint,Cosmetics Shop,Hotel
1,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Farmers Market,Steakhouse,Italian Restaurant,Cheese Shop,Café,Beer Bar
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Bar,Plane,Coffee Shop,Sculpture Garden,Boutique,Boat or Ferry
3,"Cabbagetown, St. James Town",Coffee Shop,Italian Restaurant,Bakery,Park,Pharmacy,Pizza Place,Pub,Café,Restaurant,Sandwich Place
4,Central Bay Street,Coffee Shop,Café,Middle Eastern Restaurant,Italian Restaurant,Burger Joint,Sandwich Place,Ice Cream Shop,Spa,Salad Place,Sushi Restaurant


In [44]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
DT_merged = DT
DT_merged = DT_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

DT_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Playground,Trail,Building,Yoga Studio,Dim Sum Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,0,Coffee Shop,Italian Restaurant,Bakery,Park,Pharmacy,Pizza Place,Pub,Café,Restaurant,Sandwich Place
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Coffee Shop,Gay Bar,Japanese Restaurant,Sushi Restaurant,Restaurant,Men's Store,Hotel,Fast Food Restaurant,Mediterranean Restaurant,Gastropub
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,0,Coffee Shop,Pub,Bakery,Park,Café,Restaurant,Mexican Restaurant,Breakfast Spot,Theater,Gym / Fitness Center
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Cosmetics Shop,Diner,Bakery,Plaza,Pizza Place,Bookstore


### Visualizing Clusters

In [45]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
#Adding Markers
markers_colors = []
for lat, lon, poi, cluster in zip(DT_merged['Latitude'],DT_merged['Longitude'],DT_merged['Neighbourhood'],DT_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examining Clusters

### Cluster 0

In [46]:
DT_merged.loc[DT_merged['Cluster Labels'] == 0, DT_merged.columns[[1] + list(range(5, DT_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Bakery,Park,Pharmacy,Pizza Place,Pub,Café,Restaurant,Sandwich Place
2,Downtown Toronto,0,Coffee Shop,Gay Bar,Japanese Restaurant,Sushi Restaurant,Restaurant,Men's Store,Hotel,Fast Food Restaurant,Mediterranean Restaurant,Gastropub
3,Downtown Toronto,0,Coffee Shop,Pub,Bakery,Park,Café,Restaurant,Mexican Restaurant,Breakfast Spot,Theater,Gym / Fitness Center
4,Downtown Toronto,0,Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Cosmetics Shop,Diner,Bakery,Plaza,Pizza Place,Bookstore
5,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Clothing Store,Bakery,Breakfast Spot,Cosmetics Shop,Beer Bar
6,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Farmers Market,Steakhouse,Italian Restaurant,Cheese Shop,Café,Beer Bar
7,Downtown Toronto,0,Coffee Shop,Café,Middle Eastern Restaurant,Italian Restaurant,Burger Joint,Sandwich Place,Ice Cream Shop,Spa,Salad Place,Sushi Restaurant
8,Downtown Toronto,0,Coffee Shop,Café,Steakhouse,Bar,American Restaurant,Thai Restaurant,Restaurant,Burger Joint,Cosmetics Shop,Hotel
9,Downtown Toronto,0,Coffee Shop,Hotel,Aquarium,Café,Fried Chicken Joint,Scenic Lookout,Brewery,Restaurant,Pizza Place,Sports Bar
10,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Bar,Bakery,Gastropub,Deli / Bodega,American Restaurant,Italian Restaurant


### Cluster 1

In [47]:
DT_merged.loc[DT_merged['Cluster Labels'] == 1, DT_merged.columns[[1] + list(range(5, DT_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Park,Playground,Trail,Building,Yoga Studio,Dim Sum Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


### Cluster 2

In [48]:
DT_merged.loc[DT_merged['Cluster Labels'] == 2, DT_merged.columns[[1] + list(range(5, DT_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Downtown Toronto,2,Café,Theater,Bakery,Bookstore,Sandwich Place,Restaurant,Bar,Japanese Restaurant,Noodle House,Comfort Food Restaurant
13,Downtown Toronto,2,Café,Vegetarian / Vegan Restaurant,Bar,Chinese Restaurant,Dumpling Restaurant,Vietnamese Restaurant,Bakery,Mexican Restaurant,Coffee Shop,Gaming Cafe


### Cluster 3

In [49]:
DT_merged.loc[DT_merged['Cluster Labels'] == 3, DT_merged.columns[[1] + list(range(5, DT_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,3,Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Bar,Plane,Coffee Shop,Sculpture Garden,Boutique,Boat or Ferry


### Cluster 4

In [50]:
DT_merged.loc[DT_merged['Cluster Labels'] == 4, DT_merged.columns[[1] + list(range(5, DT_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Downtown Toronto,4,Grocery Store,Café,Park,Nightclub,Diner,Restaurant,Italian Restaurant,Baby Store,Athletics & Sports,Convenience Store
